In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [11]:
import json

business_file_path = "../yelp_dataset/yelp_academic_dataset_business.json"
review_file_path = "../yelp_dataset/yelp_academic_dataset_review.json"

business_info_dict = {}
# business_info_df = pd.DataFrame()
with open(business_file_path, 'rb') as business_data:
    for b in business_data:
        b_dict = json.loads(b)

        bid = b_dict['business_id']
        category = b_dict["categories"]
        if (category is not None and "Restaurants" in category):
            business_info_dict[bid] = b_dict
#             business_info_df.append(b_dict, ignore_index=True)

# print(list(business_info_dict.items())[0])
business_info_df = pd.DataFrame(business_info_dict.values())
print(business_info_df.head())

              business_id                     name              address  \
0  6iYb2HFDywm3zjuRg0shjw      Oskar Blues Taproom         921 Pearl St   
1  tCbdrRPZA0oiIYSmHG3J0w  Flying Elephants at PDX  7000 NE Airport Way   
2  D4JtQNTI4X3KcbzacDJsMw      Bob Likes Thai Food         3755 Main St   
3  jFYIsSb7r1QeESVUnXPHBw          Boxwood Biscuit        740 S High St   
4  HPA_qyMEddpAEtFof02ixg      Mr G's Pizza & Subs        474 Lowell St   

        city state postal_code   latitude   longitude  stars  review_count  \
0    Boulder    CO       80302  40.017544 -105.283348    4.0            86   
1   Portland    OR       97218  45.588906 -122.593331    4.0           126   
2  Vancouver    BC         V5V  49.251342 -123.101333    3.5           169   
3   Columbus    OH       43206  39.947007  -82.997471    4.5            11   
4    Peabody    MA       01960  42.541155  -70.973438    4.0            39   

   is_open                                         attributes  \
0        1  {'R

In [15]:
restaurant_ids = business_info_dict.keys()
# print(restaurant_ids)
review_info_dict = {}
with open(review_file_path, 'rb') as review_data:
	for r in review_data:
		r_dict = json.loads(r)

		bid = r_dict['business_id']
		# print(bid)
		if (bid in restaurant_ids): # this is a restaurant
			if (bid in review_info_dict):
				review_info_dict[bid].append(r_dict)
			else:
				review_info_dict[bid] = [r_dict]

# review_info_df = pd.DataFrame(review_info_dict.values())
                


In [18]:
print(list(review_info_dict.items())[0])

('buF9druCkbuXLX526sGELQ', [{'review_id': 'lWC-xP3rd6obsecCYsGZRg', 'user_id': 'ak0TdVmGKo4pwqdJSTLwWw', 'business_id': 'buF9druCkbuXLX526sGELQ', 'stars': 4.0, 'useful': 3, 'funny': 1, 'cool': 1, 'text': "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four chee

In [49]:
# convert reviews to dataframe
review_info_df = pd.DataFrame(list(review_info_dict.values())[0])
count = 0
num_restaurants = 10
for key,value in list(review_info_dict.items())[1:]:
    print("Adding %d reviews", len(value))
    print(value[0])
    for review_info in value:
        review_info_df.append(review_info, ignore_index=True)
    if count >= num_restaurants:
        break
    count = count + 1
    
# review_info_df = pd.DataFrame(review_info_dict.values())
print(review_info_df.head())

Adding %d reviews 297
{'review_id': 'T5fAqjjFooT4V0OeZyuk1w', 'user_id': 'SFQ1jcnGguO0LYWnbbftAA', 'business_id': '0AzLzHfOJgL7ROwhdww2ew', 'stars': 2.0, 'useful': 1, 'funny': 1, 'cool': 1, 'text': "I've stayed at many Marriott and Renaissance Marriott's and this was a huge disappointment! The front desk and atrium is nice..there is a starbucks on site which is nice.\n\nThe rooms are run down and old.  There is a flat screen but that is to be expected of a Renaissance.\n\nWe got this hotel via Priceline at a rate of $75/night...good deal for the price but this is not a true Renaissance.", 'date': '2010-01-08 02:29:15', 'bid': '0AzLzHfOJgL7ROwhdww2ew'}
Adding %d reviews 1009
{'review_id': 'sjm_uUcQVxab_EeLCqsYLg', 'user_id': '0kA0PAJ8QFMeveQWHFqz2A', 'business_id': '8zehGz9jnxPqXtOc7KaJxA', 'stars': 4.0, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "The food is always great here. The service from both the manager as well as the staff is super. Only draw back of this restaurant is it's su

                review_id                 user_id             business_id  \
0  lWC-xP3rd6obsecCYsGZRg  ak0TdVmGKo4pwqdJSTLwWw  buF9druCkbuXLX526sGELQ   
1  hpcZLEzqD4_gPi6eSVi_Bg  Y-j2svl0M_5-jF1ehYuNPQ  buF9druCkbuXLX526sGELQ   
2  3FvY1Se8y2WXqTbaANOqMw  xUCX4GhBpeWxZB0l2lmt_w  buF9druCkbuXLX526sGELQ   
3  C1uQNP2ehBktS43ZRMEvkg  2M6KFsWIUXElqcQRz4A0Qg  buF9druCkbuXLX526sGELQ   
4  Cja8_35_kQDnF9g4voikzw  t5SRIRU6INiAyVkiMJhRPA  buF9druCkbuXLX526sGELQ   

   stars  useful  funny  cool  \
0    4.0       3      1     1   
1    2.0       1      1     1   
2    5.0       3      0     0   
3    5.0       1      2     0   
4    1.0       0      0     0   

                                                text                 date  \
0  Apparently Prides Osteria had a rough summer a...  2014-10-11 03:34:02   
1  I was really disappointed to say the least. I ...  2014-11-30 00:24:42   
2  This is as close to dining in Italy as you'll ...  2014-04-19 16:48:28   
3  Great food and service! Aga

In [50]:
# Pre Processing
# lower case
review_info_df['pre_process'] = review_info_df['text'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))

# remove html tabs
from bs4 import BeautifulSoup
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: BeautifulSoup(x).get_text())
import re
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: re.sub(r'http\S+', '', x))

# replace contractions
def contractions(s):
 s = re.sub(r'won\'t', 'will not',s)
 s = re.sub(r'would\'t', 'would not',s)
 s = re.sub(r'could\'t', 'could not',s)
 s = re.sub(r'\'d', ' would',s)
 s = re.sub(r'can\'t', 'can not',s)
 s = re.sub(r'n\'t', ' not', s)
 s= re.sub(r'\'re', ' are', s)
 s = re.sub(r'\'s', ' is', s)
 s = re.sub(r'\'ll', ' will', s)
 s = re.sub(r'\'t', ' not', s)
 s = re.sub(r'\'ve', ' have', s)
 s = re.sub(r'\'m', ' am', s)
 return s

review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x:contractions(x))

# remove non alpha chars
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: ' '.join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

# remove extra spaces
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: re.sub(' +', ' ', x))

# remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: ' '.join([x for x in x.split() if x not in stop]))

# perform lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
review_info_df['pre_process']=review_info_df['pre_process'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [51]:
review_info_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,bid,pre_process
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02,buF9druCkbuXLX526sGELQ,apparently pride osteria rough summer evidence...
1,hpcZLEzqD4_gPi6eSVi_Bg,Y-j2svl0M_5-jF1ehYuNPQ,buF9druCkbuXLX526sGELQ,2.0,1,1,1,I was really disappointed to say the least. I ...,2014-11-30 00:24:42,buF9druCkbuXLX526sGELQ,really disappointed say least saw really great...
2,3FvY1Se8y2WXqTbaANOqMw,xUCX4GhBpeWxZB0l2lmt_w,buF9druCkbuXLX526sGELQ,5.0,3,0,0,This is as close to dining in Italy as you'll ...,2014-04-19 16:48:28,buF9druCkbuXLX526sGELQ,close dining italy find new england chef paolo...
3,C1uQNP2ehBktS43ZRMEvkg,2M6KFsWIUXElqcQRz4A0Qg,buF9druCkbuXLX526sGELQ,5.0,1,2,0,Great food and service! Again. 4 out of the la...,2014-09-10 00:06:00,buF9druCkbuXLX526sGELQ,great food service last week great every time ...
4,Cja8_35_kQDnF9g4voikzw,t5SRIRU6INiAyVkiMJhRPA,buF9druCkbuXLX526sGELQ,1.0,0,0,0,We ordered the roasted chicken and homemade pa...,2014-08-24 23:53:23,buF9druCkbuXLX526sGELQ,ordered roasted chicken homemade pasta w pesto...


In [52]:
# tfidf
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train, Y_test = train_test_split(review_info_df['pre_process'], review_info_df['review_id'], test_size=0.25, random_state=30)

print('Train: ',X_train.shape,Y_train.shape,'Test: ',(X_test.shape,Y_test.shape))

Train:  (63,) (63,) Test:  ((21,), (21,))


In [48]:
print('TFIDF Vectorizer...')
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer...
